# Классификация токсичных комментариев

## 1. Подготовка данных

### 1.1 Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook, tqdm
import catboost
import re
import nltk
import warnings


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from transformers import AutoModel, AutoTokenizer 


from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
stopwords = nltk_stopwords.words('english')

warnings.filterwarnings('ignore')

In [2]:
#проверка на наличие GPU-устройства
torch.cuda.is_available()

True

In [3]:
#название устройства
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

### 1.2 Загрузка данных

In [4]:
#Загрузка данных
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('C:/Users/XE/Downloads/toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
#просмотр информации о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


 159292 обьектов - тексты комментариев и метка токсичный или нет. Unnamed - лишний столбец с индексами.

In [6]:
#удаление первого столбца, который не несет никакой полезной информации
df.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
#проверка на дубликаты
df.duplicated().sum()

0

In [8]:
#соотношение таргетов
df.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Наблюдается дисбаланс классов, нетоксичных комментариев в 9 раз меньше, чем токсичных.

## 2. BERT

### 2.1 Загрузка BERT

В данной работе я буду использовать специализированный BERT, взятый с https://huggingface.co/unitary/toxic-bert

In [9]:
#Загрузка модели и токенайзера toxic-bert
model_name = "unitary/toxic-bert" 


model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Для передачи модели текстов их нужно токенизировать, для этого применю к ним загруженный ранее токенайзер.

In [10]:
%%time
#токенизирую наш набор текстов
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, max_length=512, truncation=True))

CPU times: total: 32.4 s
Wall time: 32.4 s


In [11]:
#так выглядит токенизированный датасет
tokenized

0         [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1         [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2         [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3         [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4         [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                                ...                        
159287    [101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...
159288    [101, 2017, 2323, 2022, 14984, 1997, 4426, 200...
159289    [101, 13183, 6290, 26114, 1010, 2045, 2015, 20...
159290    [101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...
159291    [101, 1000, 1998, 1012, 1012, 1012, 1045, 2428...
Name: text, Length: 159292, dtype: object

Затем необходимо обрезать все предложения до 512 слов - максимальная длина вектора у модели, а также создать attention mask: вектор с нулями и единицами, задача которого сообщить модели на какие токены обращать внимание (1), а на какие — нет (0).

In [12]:
#padding
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

#attention_mask
attention_mask = np.where(padded != 0, 1, 0)

In [13]:
#чтобы уложиться в нашу оперативную память поделю датасет на батчи по 100 объектов
#эмбеддинги будут сохраняться в переменную embeddings
embeddings = []
batch_size = 100

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device);

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/1592 [00:00<?, ?it/s]

In [14]:
#обьединение ембеддингов в признаки и выделение целевого признака
#количество таргетов приведу в соответствие с признаками, т.к. я делил их на батчи по 100 шт.
X = np.concatenate(embeddings)
y = df['toxic'][:len(X)]

### 2.2 Деление на выборки

In [15]:
#поделю выборки на обучающую и тестовую в соотношении 80:20 со стратификацией по целевому признаку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

### 2.3 Обучение моделей

**Разбиение на фолды**

In [16]:
#стратифицирую разбиение на фолды
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=777)

#### 2.3.1 Baseline

In [17]:
#посмотрю чему будет равна f1-мера при заполнении всех значений единицей
f1_score(y_test, np.ones(len(y_test)))

0.18451362755160225

#### 2.3.2 LogisticRegression

In [18]:
%%time
#обучение модели логистической регрессии с использованием GridSearchCV
log = LogisticRegression(random_state=777)
params = {'penalty': ['l1', 'l2', 'elasticnet'],
         'C': [0.001, 0.01, 0.1, 1, 5],
         'class_weight': [None, 'balanced']}
log_grid = GridSearchCV(log, params, n_jobs=-1, cv = skf, scoring='f1', verbose=20)
log_grid.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
CPU times: total: 10.8 s
Wall time: 2min 33s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=777, shuffle=True),
             estimator=LogisticRegression(random_state=777), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 5],
                         'class_weight': [None, 'balanced'],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='f1', verbose=20)

In [19]:
print('f1 метрика для модели логистической регрессии', log_grid.best_score_)

f1 метрика для модели логистической регрессии 0.9479105471962699


In [20]:
print('Лучшие параметры для модели логистической регрессии', log_grid.best_params_)

Лучшие параметры для модели логистической регрессии {'C': 0.1, 'class_weight': None, 'penalty': 'l2'}


#### 2.3.3 SGD

In [21]:
%%time
#обучение SGD с использованием GridSearchCV
sgd = SGDClassifier(random_state=777)
params = {'penalty': ['l2', 'elasticnet'],
         'alpha': [0.000001, 0.00001, 0.001, 0.1, 1, 10],
         'class_weight': [None, 'balanced']}
sgd_grid = GridSearchCV(sgd, params, n_jobs=-1, cv = skf, scoring='f1', verbose=20)
sgd_grid.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
CPU times: total: 2.83 s
Wall time: 1min 31s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=777, shuffle=True),
             estimator=SGDClassifier(random_state=777), n_jobs=-1,
             param_grid={'alpha': [1e-06, 1e-05, 0.001, 0.1, 1, 10],
                         'class_weight': [None, 'balanced'],
                         'penalty': ['l2', 'elasticnet']},
             scoring='f1', verbose=20)

In [22]:
print('f1 метрика для SGD', sgd_grid.best_score_)

f1 метрика для SGD 0.9459726842527506


In [23]:
print('параметры для sgd', sgd_grid.best_params_)

параметры для sgd {'alpha': 0.001, 'class_weight': None, 'penalty': 'l2'}


#### 2.3.4 Catboost

In [24]:
%%time
#обучение catboost classifier
cat = catboost.CatBoostClassifier(random_state=777, task_type="GPU", verbose=False)
params = {
    'iterations': [100, 500, 1000],
    'depth': [4, 5, 6],
    'learning_rate': [0.01, 0.03, 0.1],
    'min_child_samples': [1, 4, 8]
}
cat_grid = GridSearchCV(cat, params, cv=skf, scoring='f1', verbose=0)

CPU times: total: 0 ns
Wall time: 13 ms


In [ ]:
cat_grid.fit(X_train, y_train)

In [26]:
print('f1 метрика catboost', cat_grid.best_score_)

f1 метрика catboost 0.9468908012326276


In [27]:
print('Лучшие параметры catboost', cat_grid.best_params_)

Лучшие параметры catboost {'depth': 6, 'iterations': 1000, 'learning_rate': 0.01, 'min_child_samples': 1}


**Выводы**:
1. Все модели показали близкий результат (f1-мера около 0.94) и достигли искомого значения метрики качества(0.75 f1);
2. Лучшая модель по скорости обучения SGD;
3. Лучшая модель - Логистическая регрессия (0.948 f1 на кросс-валидации).

## 3. TF-IDF

### 3.1 Очистка и лемматизация

Для выполнения векторизации с помощью TF-IDF необходимо произвести очистку текстов и ее лемматизацию

In [28]:
#функция очистки
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text.lower()).split())

In [29]:
#функция распознавания части речи
def get_wordnet_pos(word):
    treebank_tag = nltk.pos_tag([word])[0][1][0].upper()
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

In [30]:
#функция лемматизации 
def lemmatize(text):
    lm = nltk.WordNetLemmatizer()
    return ' '.join([lm.lemmatize(word, get_wordnet_pos(word)) for word in nltk.word_tokenize(text)])

In [31]:
#очистка текста
df['clear'] = df['text'].apply(clear_text)
df.clear.head()

0    explanation why the edits made under my userna...
1    d aww he matches this background colour i m se...
2    hey man i m really not trying to edit war it s...
3    more i can t make any real suggestions on impr...
4    you sir are my hero any chance you remember wh...
Name: clear, dtype: object

In [32]:
#проверка на работоспособность
lemmatize(clear_text('111bats cats?!?!? dogs'))

'bat cat dog'

In [33]:
#лемматизация текста
lemmatized_text = []
for i in tqdm(df['clear'].values):
    lemmatized_text.append(lemmatize(i))

100%|██████████████████████████████████████████████████████████████████████████| 159292/159292 [52:17<00:00, 50.77it/s]


Так выглядит лемматизированный текст:

In [34]:
lemmatized_text[0]

'explanation why the edits make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after i vote at new york doll fac and please don t remove the template from the talk page since i m retire now'

### 3.2 Деление на выборки

In [35]:
#поделю выборки на обучающую и тестовую в соотношении 80:20
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(
    lemmatized_text, df['toxic'], test_size=0.2, stratify=df['toxic'], random_state=777)

### 3.3 Обучение моделей

Так как обучение достаточно ресурсозатратное в данной работе буду обучать только линейные модели.

Во избежании утечки в процессе векторизации TF-IDF буду использовать конвееры pipeline.

#### 3.3.1 LogisticRegression

In [36]:
#logistic pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=777)),
])

parameters = [{
    'vect__max_df': (0.5, 1.0),
    'vect__ngram_range': ((1, 1), (1, 2), (1,3), (1, 4)),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__C': (0.1, 1, 5)
}]

log_grid_tf = GridSearchCV(pipeline, parameters, cv=skf, n_jobs=-1, scoring='f1')

In [37]:
%%time
log_grid_tf = log_grid_tf.fit(X_train_tf, y_train_tf)

CPU times: total: 1min 52s
Wall time: 36min 16s


In [38]:
print('Лучшая метрика для logistic regression', log_grid_tf.best_score_)

Лучшая метрика для logistic regression 0.7657400723919409


In [39]:
print('Лучшие параметры для logistic regression', log_grid_tf.best_params_)

Лучшие параметры для logistic regression {'clf__C': 5, 'clf__penalty': 'l2', 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}


#### 3.3.2 SGD

In [40]:
#sgd in pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words=stopwords)),
    ('clf', SGDClassifier(random_state=777)),
])

parameters = [{
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (1, 2), (1,3), (1, 4)),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__class_weight': ['balanced', None]
}]

sgd_grid_tf = GridSearchCV(pipeline, parameters, cv=skf, n_jobs=-1, scoring='f1')

In [41]:
%%time
sgd_grid_tf =sgd_grid_tf.fit(X_train_tf, y_train_tf)

CPU times: total: 2min 25s
Wall time: 20min 11s


In [42]:
print('Лучшая метрика для SGD', sgd_grid_tf.best_score_)

Лучшая метрика для SGD 0.7890070796964928


In [43]:
print('Лучшие параметры для SGD', sgd_grid_tf.best_params_)

Лучшие параметры для SGD {'clf__alpha': 1e-06, 'clf__class_weight': None, 'clf__penalty': 'elasticnet', 'vect__max_df': 0.5, 'vect__ngram_range': (1, 4)}


TF-IDF векторизация также справилась с задачей - удалось построить модель, которая показывает необходимое значение f1-меры (0.75). 

## 4. Тестирование моделей

In [50]:
print('f1-мера на тестовой выборке SGD(BERT)', f1_score(y_test, sgd_grid.predict(X_test)))

f1-мера на тестовой выборке SGD(BERT) 0.9454155167126648


In [45]:
print('f1-мера на тестовой выборке SGD(TF-IDF)', f1_score(y_test_tf, sgd_grid_tf.predict(X_test_tf)))

f1-мера на тестовой выборке SGD(TF-IDF) 0.7895668974400257


### Выводы:

1. Данные были загружены и исследованы.
2. Данные были токенизированы с помощью Bert-tokenizer. Были построены эмбеддинги с помощью BERT model. Затем данные были разбиты на обучающие и тестовые выборки. Были обучены три модели - логистическая регрессия, SGD и catboost. Все модели достигли искомой метрики качества, лучший результат у логистической регрессии: 0.947. Лучшая модель по скорости обучения - SGD.
3. Данные отдельно были очищены и лемматизированы. Данные были разбиты на обучающую и тестовые выборки, обучающая выборка была векторизирована TfidfVectorizer и обучена с использованием конвееров и GridSearchCV для кросс-валидации. Были обучены две линейных модели - Logistic Regression и SGD. Обе модели показали близкое значение метрики в районе 0.78, при этом обучение SGD происходило значительно быстрее.
4. Была отобрана модель для тестирования - SGD, данная модель обладает меньшей ресурсозатратностью при обучении и подборе параметров, при близком к логистической регрессии значении метрики качества.
5. Модели были протестированы на тестовой выборке, удалось достичь результат выше 0.75 f1-меры - лучше всего себя показала модель с BERT (f1-мера 0.945 на тестовой выборке).